In [148]:
# Download imprtant packages
import numpy as np
import pandas as pds
import random
import copy
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam, RMSprop
from collections import deque
from keras import backend as K

In [149]:
random.seed(30) #############################generate a random maze, with size 10 by 10.
class Maze(object):# ########################this is the class maze object, i.e. we create a random maze
    def __init__(self, size=10, blocks_rate=0.2):
        self.size = size if size > 3 else 10
        self.blocks = int((size ** 2) * blocks_rate) 
        self.s_list = []
        self.maze_list = []
        self.e_list = []

    def create_mid_lines(self, k):#########This is where we create hedges in the maze, i.e. 'X'
        if k == 0: self.maze_list.append(self.s_list)
        elif k == self.size - 1: self.maze_list.append(self.e_list)
        else:
            tmp_list = []
            for l in range(0,self.size):
                if l == 0: tmp_list.extend("X")
                elif l == self.size-1: tmp_list.extend("X")
                else:
                    a = random.randint(-1, 0)####### generate random -1 and 0 rewards
                    tmp_list.extend([a])
            self.maze_list.append(tmp_list)

    def insert_blocks(self, k, s_r, e_r):######inserting 'x' into the maze.
        b_y = random.randint(1, self.size-2)
        b_x = random.randint(1, self.size-2)
        if [b_y, b_x] == [1, s_r] or [b_y, b_x] == [self.size - 2, e_r]: k = k-1
        else: self.maze_list[b_y][b_x] = "X"
            
    def generate_maze(self): ######## this sets up entry cell to maze and assigns '#'
        s_r = random.randint(1, (self.size / 2) - 1)
        for i in range(0, self.size):
            if i == s_r: self.s_list.extend("#")
            else: self.s_list.extend("X")
        start_point = [0, s_r]

        e_r = random.randint((self.size / 2) + 1, self.size - 2)
        for j in range(0, self.size):
            if j == e_r: self.e_list.extend([100])
            else: self.e_list.extend("X")
        goal_point = [self.size - 1, e_r]

        for k in range(0, self.size):
            self.create_mid_lines(k)
        
        for k in range(self.blocks):
            self.insert_blocks(k, s_r, e_r)

        return self.maze_list, start_point, goal_point
random.seed(30)#############################################this is done to reproduce maze.

In [150]:
random.seed(30)############this is where we introduce agent and reward system 
class Field(object):
    def __init__(self, maze, start_point, goal_point):
        self.maze = maze
        self.start_point = start_point
        self.goal_point = goal_point
        self.movable_vec = [[1,0],[-1,0],[0,1],[0,-1]]

    def display(self, point=None):
        field_data = copy.deepcopy(self.maze)
        if not point is None:
                y, x = point
                field_data[y][x] = "@"
        else:
                point = ""
        for line in field_data:
                print ("\t" + "%3s " * len(line) % tuple(line))

    def get_actions(self, state):####### this defines actions agent can make, it should avoid 'x' 
        movables = []
        if state == self.start_point:
            y = state[0] + 1
            x = state[1]
            a = [[y, x]]
            return a
        else:
            for v in self.movable_vec:
                y = state[0] + v[0]
                x = state[1] + v[1]
                if not(0 < x < len(self.maze) and
                       0 <= y <= len(self.maze) - 1 and
                       maze[y][x] != "X" and
                       maze[y][x] != "#"):
                    continue
                movables.append([y,x])
            if len(movables) != 0:
                return movables
            else:
                return None

    def get_val(self, state):
        y, x = state
        if state == self.start_point: return 0, False
        else:
            v = float(self.maze[y][x])
            if state == self.goal_point: 
                return v, True
            else: 
                return v, False

random.seed(30)

In [151]:
random.seed(30)# this builds the maze and displays it
size = 10
barriar_rate = 0.2

maze_1 = Maze(size, barriar_rate)
maze, start_point, goal_point = maze_1.generate_maze()
maze_field = Field(maze, start_point, goal_point)

maze_field.display()
random.seed(30)

	  X   X   X   #   X   X   X   X   X   X 
	  X  -1  -1   0   X   X   0  -1  -1   X 
	  X   0  -1  -1  -1  -1   X   0   0   X 
	  X  -1   0   X  -1   0   0   X   X   X 
	  X  -1   0   0  -1   0   0   0  -1   X 
	  X  -1  -1   0  -1  -1   X   0   X   X 
	  X  -1   0   0   0   X   0  -1  -1   X 
	  X   X   X   0   0   X   0   X   X   X 
	  X   X  -1   X   0  -1  -1   0   0   X 
	  X   X   X   X   X   X   X   X 100   X 


In [152]:
random.seed(30)
class QLearning_Solver(object):########## this builds Q-learning algorithm and defines all the parameters, such alpha and so on...
    def __init__(self, maze, display=False):
        self.Qvalue = {}
        self.Field = maze
        self.alpha = 0.6
        self.gamma  = 0.7
        self.epsilon = 0.8
        self.steps = 0
        self.score = 0
        self.display = display

    def qlearn(self, greedy_flg=False):####### this defines the reward system and how it is accumulated and also defines the step sizes 
        state = self.Field.start_point 
        while True:
            if greedy_flg:
                self.steps += 1
                action = self.choose_action_greedy(state)
                print("current state: {0} -> action: {1} ".format(state, action))# this sets the states and action and displays it
                if self.display:
                    self.Field.display(action)
                reward, tf = self.Field.get_val(action)
                self.score =  self.score + reward
                print("current step: {0} \t score: {1}\n".format(self.steps, self.score))
                if tf == True:
                    print("Goal!")############# print goal once algorithm converges for greedy policy
                    break
            else:
                action = self.choose_action(state)    
            if self.update_Qvalue(state, action):
                break
            else:
                state = action

    def update_Qvalue(self, state, action):#### defines and calculates the Q-learning algorithm
        Q_s_a = self.get_Qvalue(state, action)
        mQ_s_a = max([self.get_Qvalue(action, n_action) for n_action in self.Field.get_actions(action)])
        r_s_a, finish_flg = self.Field.get_val(action)
        q_value = Q_s_a + self.alpha * ( r_s_a +  self.gamma * mQ_s_a - Q_s_a)
        self.set_Qvalue(state, action, q_value)
        return finish_flg


    def get_Qvalue(self, state, action):### get q-value for each state and action pair
        state = (state[0],state[1])
        action = (action[0],action[1])
        try:
            return self.Qvalue[state][action]
        except KeyError:
            return 0.0

    def set_Qvalue(self, state, action, q_value):
        state = (state[0],state[1])
        action = (action[0],action[1])
        self.Qvalue.setdefault(state,{})
        self.Qvalue[state][action] = q_value

    def choose_action(self, state):### define and implement epsilon/greedy policy
        if self.epsilon < random.random():
            return random.choice(self.Field.get_actions(state))
        else:
            return self.choose_action_greedy(state)

    def choose_action_greedy(self, state):
        best_actions = []
        max_q_value = -50
        for a in self.Field.get_actions(state):
            q_value = self.get_Qvalue(state, a)
            if q_value > max_q_value:
                best_actions = [a,]
                max_q_value = q_value
            elif q_value == max_q_value:
                best_actions.append(a)
        return random.choice(best_actions)

    def dump_Qvalue(self):########print each Q-value for each state and action
        print("##### Dump Qvalue #####")
        for i, s in enumerate(self.Qvalue.keys()):
            for a in self.Qvalue[s].keys():
                print("\t\tQ(s, a): Q(%s, %s): %s" % (str(s), str(a), str(self.Qvalue[s][a])))
            if i != len(self.Qvalue.keys())-1: 
                print('\t------------------state   action   reward')
        
random.seed(30)

In [153]:
random.seed(30)
learning_count = 8 ### epoch, how many run through agent makes in the maze
QL_solver = QLearning_Solver(maze_field, display=True)
for i in range(learning_count):
    QL_solver.qlearn()

QL_solver.dump_Qvalue()

random.seed(30)

##### Dump Qvalue #####
		Q(s, a): Q((0, 3), (1, 3)): -1.059101797404672
	------------------state   action   reward
		Q(s, a): Q((1, 3), (2, 3)): -1.6291077605375999
		Q(s, a): Q((1, 3), (1, 2)): -1.6314128161075199
	------------------state   action   reward
		Q(s, a): Q((2, 3), (1, 3)): -0.997482544128
		Q(s, a): Q((2, 3), (2, 4)): -1.2888
		Q(s, a): Q((2, 3), (2, 2)): -0.98976
	------------------state   action   reward
		Q(s, a): Q((1, 2), (1, 3)): -1.025075743888896
		Q(s, a): Q((1, 2), (1, 1)): -1.04184
		Q(s, a): Q((1, 2), (2, 2)): -0.9743999999999999
	------------------state   action   reward
		Q(s, a): Q((1, 1), (2, 1)): -0.4536
		Q(s, a): Q((1, 1), (1, 2)): -0.9144576677759999
	------------------state   action   reward
		Q(s, a): Q((2, 1), (3, 1)): -0.84
		Q(s, a): Q((2, 1), (1, 1)): -0.84
		Q(s, a): Q((2, 1), (2, 2)): -0.9359999999999999
	------------------state   action   reward
		Q(s, a): Q((3, 1), (3, 2)): 0.0
		Q(s, a): Q((3, 1), (2, 1)): -0.252
		Q(s, a): Q((3, 1), (4, 1)

In [154]:
QL_solver.qlearn(greedy_flg=True)#####implement greedy policy and display

current state: [0, 3] -> action: [1, 3] 
	  X   X   X   #   X   X   X   X   X   X 
	  X  -1  -1   @   X   X   0  -1  -1   X 
	  X   0  -1  -1  -1  -1   X   0   0   X 
	  X  -1   0   X  -1   0   0   X   X   X 
	  X  -1   0   0  -1   0   0   0  -1   X 
	  X  -1  -1   0  -1  -1   X   0   X   X 
	  X  -1   0   0   0   X   0  -1  -1   X 
	  X   X   X   0   0   X   0   X   X   X 
	  X   X  -1   X   0  -1  -1   0   0   X 
	  X   X   X   X   X   X   X   X 100   X 
current step: 1 	 score: 0.0

current state: [1, 3] -> action: [2, 3] 
	  X   X   X   #   X   X   X   X   X   X 
	  X  -1  -1   0   X   X   0  -1  -1   X 
	  X   0  -1   @  -1  -1   X   0   0   X 
	  X  -1   0   X  -1   0   0   X   X   X 
	  X  -1   0   0  -1   0   0   0  -1   X 
	  X  -1  -1   0  -1  -1   X   0   X   X 
	  X  -1   0   0   0   X   0  -1  -1   X 
	  X   X   X   0   0   X   0   X   X   X 
	  X   X  -1   X   0  -1  -1   0   0   X 
	  X   X   X   X   X   X   X   X 100   X 
current step: 2 	 score: -1.0

current state: [2